In [127]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [128]:
# load dataset
data = pd.read_csv('Ozone Level Detection.data',
                   sep=";|:|,",
                   header=None,
                   engine='python')

# change missing value to np.nan
data[data == '?'] = np.nan

#data.isnull().sum()
np_data = data.to_numpy()
np_data.shape
#data

(2536, 74)

#رفع داده های گم‌شده

برای رفع داده های گم شده از منبع زیر استفاده شده است.
https://www.analyticsvidhya.com/blog/2021/10/handling-missing-value/


Missing values are imputed using the k-Nearest Neighbors approach where a Euclidean distance is used to find the nearest neighbors.

In [129]:
X = data.iloc[:,1:74]

In [130]:
from sklearn.impute import KNNImputer
impute_knn = KNNImputer(n_neighbors=5)
FixedInputData = impute_knn.fit_transform(X)
FixedInputData = pd.DataFrame(FixedInputData)
#FixedInputData.insert(0,-1, np_data[:,0]) Adding date as property
FixedInputData = FixedInputData.to_numpy()



## implementing KFold

ورودی این تابع تعداد تقسیم، آرایه و اینکه رندوم بشود یا نه را می گیرد و خروجی آن آرایه ای از کلاس فولد می دهد که شامل داده ی آموزش و تست است. بدیهی است که تعداد این آرایه برابر تعداد تقسیم که در ورودی گرفته است، می باشد.

In [131]:
class Fold:
    def __init__(self,X_Train,Y_Train,X_Test,Y_Test):
        self.X_Train = X_Train
        self.Y_Train = Y_Train
        self.X_Test = X_Test
        self.Y_Test = Y_Test


def KFold(NumberOfSplits, InputData, Shuffle=False):
    if(Shuffle):
         np.random.shuffle(InputData)
    output = []

    SplitedArray = np.array_split(InputData, NumberOfSplits)
    for i in range(NumberOfSplits):
        test = SplitedArray[i]
        train = np.concatenate( SplitedArray[0:i] +  SplitedArray[i + 1 :], axis=0 )
        X_Train = preprocessing.normalize(train[:,:-1])
        Y_Train = train[:, -1]

        X_Test = preprocessing.normalize(test[:,:-1])
        Y_Test = test[:,-1]
        output.append(Fold(X_Train,Y_Train,X_Test,Y_Test))
    
    return output
 

    

In [132]:
kfold_Out = KFold(3,FixedInputData)
kfold_Out[0].X_Train.shape[0]

1690

In [133]:
def sigmoid(x):
    return (1/(1 +  np.e ** (-x)))

In [134]:
def CrossEntropyLossFunction(M, Y_Hat, Y):
    return (-1/M) * (Y.dot( np.log(Y_Hat).T) + (1-Y).dot( np.log(1 - Y_Hat).T))


In [135]:
def CalculateYHat(W,X,b):
    return np.dot(W , np.transpose(X)) + b

In [136]:
def CalculateAnswer(Y_Hat):
    for i in range(Y_Hat.shape[1]):
        if(Y_Hat[0,i] > 0.5):
            Y_Hat[i] = 1
        else:
            Y_Hat[0,i] = 0
    
    return Y_Hat


In [139]:
nr_split = 3
kfold_Out = KFold(nr_split,FixedInputData,True)
kfold_Out[0].X_Train.shape[0]


learningRate = 0.1
nr_epoch = 500

for i in range (nr_split): #TODO
    # for each split do the training
    X = kfold_Out[i].X_Train
    Y = np.reshape( kfold_Out[i].Y_Train,(1, kfold_Out[i].Y_Train.shape[0]))
    #print("shape: ",X.shape)
    M = X.shape[0]
    W = np.zeros((1 ,X.shape[1])) # 1 * 72
    b = np.zeros((1,1)) # 1 * 1

    for j in range(nr_epoch): # traning for nr_epoch
            
        Z = CalculateYHat(W,X,b)  # W => 1 * 72 ||||| X => M * 72 
        
        A = sigmoid(Z) # Y_hat
        #print(A)
        L = CrossEntropyLossFunction(M,A, Y)
        
        # dL/dZ
        dZ = A - Y 
        
        
        # Calculate derivitives
        dW = (1/M) * (np.dot(X.T,dZ.T)) # dL/dW
        
        db = (1/M) * np.sum(dZ) # dL/db

        # update the weights and bias
        W = W - learningRate * np.transpose(dW)
        
        b = b - learningRate * np.transpose(db)

    # Calculate loss for current split
    X_test = kfold_Out[i].X_Test
    Y_test = np.reshape( kfold_Out[i].Y_Test,(1, kfold_Out[i].Y_Test.shape[0]))
    Z = CalculateYHat(W,X_test,b)
    Y_hat = sigmoid(Z)
    
    Y_hat = np.round(Y_hat)
    print((Y_hat == Y_test).sum() / Y_test.shape[1], "num: ", Y_test.shape)
    

0.9657210401891253 num:  (1, 846)
0.9727810650887574 num:  (1, 845)
0.9751479289940829 num:  (1, 845)
